In [1]:
import torch
import torch.cuda
import numpy as np
from numba import cuda

In [2]:
@cuda.jit
def dot_product(v1, v2, product):
    pos = cuda.grid(1)
    if pos < len(product):
        product[pos] = v1[pos] * v2[pos]

In [8]:
sms = 24
threads = 256
blocks = sms * 24

n = 500

product_vec_global_mem = cuda.device_array(n)
v1 = np.random.randint(50, size=n)
v2 = np.random.randint(50, size=n)
v1_global_mem = cuda.to_device(v1)
v2_global_mem = cuda.to_device(v2)

with torch.cuda.device(0):
    dot_product[blocks, threads](v1_global_mem, v2_global_mem, product_vec_global_mem)

product_vec = product_vec_global_mem.copy_to_host()
product = sum(product_vec)

print(product)

285958.0
